# 第３回 認識技術特論 課題答案
## 23vr008n 三宅研 高林秀



# 課題内容

- 犬と猫のデータセット演習用 Jupyter Notebook ファイル
（TransferCNN.ipynb）をまねて学習済み VGG19、ResNet などのモデルのファ
インチューニングをすることで犬と猫のデータセットを認識する code を作成
してください。
- 学習用データの水増しがありとなしを用いるとき学習した犬と
猫の認識モデルにおいて、テスト用（val フォールダーの中の画像）データの
認識結果を比較してください。

1. Jupyter Notebook で作成し、それぞれのコードについて簡単な説明を加え
てください
2. 一部の画像と認識ラベルを表示してください。
3. 考察や自分の考え方を簡単に纏めてください。

# 解答コード

In [17]:
# 共通モジュールのインポート

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms, utils
from torchvision.models import vgg19
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import os
from enum import Enum
import copy

In [13]:
# データセットの準備
class DataKeys(Enum):
    TRAIN = "train"
    VALID = "val"

DATA_DIR = "./DogCat"
TRAIN_DIR = os.path.join(DATA_DIR, "train")
VALID_DIR = os.path.join(DATA_DIR, "val")

# 画像データの前処理
data_transfroms = { 
    DataKeys.TRAIN: transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor() #テンソルへの変換
    ]),
    DataKeys.VALID: transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
}

datasets = {
    DataKeys.TRAIN: datasets.ImageFolder(TRAIN_DIR, transform=data_transfroms[DataKeys.TRAIN]),
    DataKeys.VALID: datasets.ImageFolder(VALID_DIR, transform=data_transfroms[DataKeys.VALID])
}

dataloaders = {
    DataKeys.TRAIN: DataLoader(datasets[DataKeys.TRAIN], batch_size=32, shuffle=True, num_workers=4),
    DataKeys.VALID: DataLoader(datasets[DataKeys.VALID], batch_size=32, shuffle=False, num_workers=4)
}



In [14]:
# データセットの確認

dataset_sizes = {x: len(datasets[x]) for x in [DataKeys.TRAIN, DataKeys.VALID]}
class_names = datasets[DataKeys.TRAIN].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Dataset size: {dataset_sizes}")
print(f"Train Class names: {class_names}")
print(f"Validate Class names:{datasets[DataKeys.VALID].classes}")
print(f"Learning Device: {device}")


Dataset size: {<DataKeys.TRAIN: 'train'>: 2200, <DataKeys.VALID: 'val'>: 544}
Train Class names: ['Cat', 'Dog']
Validate Class names:['Cat', 'Dog']
Learning Device: cuda:0


In [18]:
# ランダムに5枚の画像を表示する関数
def imshow(inp, title=None):
    inp = inp.cpu().numpy().transpose((1, 2, 0))  # CPUテンソルに変換してからNumpy配列に変換
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # プロットが更新されるのを一時停止

# ランダムに5枚の画像を表示
def show_random_images(dataloader):
    inputs, classes = next(iter(dataloader))
    out = utils.make_grid(inputs[:5])
    imshow(out, title=[class_names[x] for x in classes[:5]])
    plt.show()

# 訓練データ中任意の5枚を表示
show_random_images(dataloaders[DataKeys.TRAIN])
plt.title("Training DataSets")
plt.show()


RuntimeError: Numpy is not available

In [ ]:
# 検証データ中任意の5枚を表示
inputs, classes = next(iter(dataloaders[DataKeys.VALID]))
out = torchvision.utils.make_grid(inputs[:5])
imshow(out, title=[class_names[x] for x in classes[:5]])
plt.title("Validation DataSets")
plt.show()

In [ ]:
# モデルのトレーニング関数
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # モデルをトレーニングモードに設定
            else:
                model.eval()   # モデルを評価モードに設定

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# テスト用関数
def test_model(model, criterion, dataloader):
    model.eval()  # モデルを評価モードに設定

    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / dataset_sizes['val']
    acc = running_corrects.double() / dataset_sizes['val']

    print(f'Test Loss: {loss:.4f} Acc: {acc:.4f}')

## 学習データの水増し無しのパターン

In [ ]:
model = vgg19(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2) # 2値分類に変更

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## 学習データの水増しありのパターン

## 両パターンの比較と考察